In [1]:
import pandas as pd

import gym
import numpy as np
from stable_baselines3 import A2C, DDPG, SAC, PPO
from matplotlib import axis
import numpy as np
from floris.tools import FlorisInterface
from floris.tools.visualization import visualize_cut_plane
import matplotlib.pyplot as plt
from floris.tools.optimization.yaw_optimization.yaw_optimizer_sr import YawOptimizationSR

#Importing custom environment.
from Custom_functions import CustomEnv

In [2]:
df = pd.read_excel (r'Results\Overview.xlsx')

In [3]:
model_compare = "PPO3"

In [4]:
index = df["Model name"].str.find(model_compare)[0]

In [5]:
Model_name = df["Model name"][index]
t_steps = df["t_steps"][index]
dist = df["dist"][index]
nx = df["nx"][index]
ny = df["ny"][index]
turb_type = df["turb_type"][index]
combination = df["combination"][index]
deflection = df["deflection"][index]
turbulence = df["turbulence"][index]
velocity = df["velocity"][index]
VS_min = df["VS_min"][index]
VS_max = df["VS_max"][index]
TI_min = df["TI_min"][index]
TI_max = df["TI_max"][index]
wd_min = df["wd_min"][index]
wd_max = df["wd_max"][index]
yaw_max = df["yaw_max"][index]
rho = df["rho"][index]

In [6]:
env = CustomEnv(t_steps, dist, nx, ny, turb_type, combination, deflection,
               turbulence, velocity, VS_min, VS_max, TI_min, TI_max, wd_min, wd_max, 
               yaw_max, rho)

In [8]:
model_path = "models/"+model_compare+"/720000"

In [9]:
model = PPO.load(model_path, env=env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [115]:
#Creates the base for the farm
fi = FlorisInterface("gch.yaml")   

#Turns it into a dictionary and then does the changes to the model
fi_dict = fi.floris.as_dict()

fi_dict["farm"]["turbine_type"] = [turb_type]
fi_dict["wake"]["model_strings"]["combination_model"] = combination
fi_dict["wake"]["model_strings"]["deflection_model"]  = deflection
fi_dict["wake"]["model_strings"]["turbulence_model"]  = turbulence
fi_dict["wake"]["model_strings"]["velocity_model"]    = velocity
fi_dict["flow_field"]["air_density"]                  = rho

# Turns it back into a floris object:
fi = FlorisInterface(fi_dict)

D = fi.floris.farm.rotor_diameters[0]

x = np.linspace(0, D*dist*nx, nx)
y = np.linspace(0, D*dist*ny, ny)

xv, yv = np.meshgrid(x, y, indexing='xy')

layout_x =  xv.flatten()
layout_y = yv.flatten()

fi.reinitialize(
            layout=(layout_x, layout_y),
            )


In [122]:
TI = TI_min

actions = []
agent_power = []
baseline_power = []

yaw_angles = np.zeros((1, 1, nx*ny)) 
yaw_zero = np.zeros((1, 1, nx*ny)) 

for wind_speed in range(VS_min,VS_max+1):
    #print(wind_speed)
    for wind_dir in range(wd_min, wd_max+1):
        observation = np.array([wind_speed, TI, wind_dir], dtype = np.float32)
        action, _states = model.predict(observation, deterministic=True)
        
        action = action*yaw_max   #scales the action to the 
        actions.append(action)
        
        fi.reinitialize(
            wind_directions=[wind_dir],
            turbulence_intensity= TI,
            wind_speeds=[wind_speed]
            )
        
        yaw_angles[0,0,:] = action
        fi.calculate_wake(yaw_angles = yaw_zero)
        
        baseline_power.append(fi.get_farm_power()[0][0])
        
        fi.calculate_wake(yaw_angles = yaw_angles)
        agent_power.append(fi.get_farm_power()[0][0])
        
        

actions = np.array(actions)
np.unique(actions,axis = 0)

baseline_power = np.array(baseline_power)
agent_power = np.array(agent_power)


In [125]:
diff = agent_power - baseline_power

In [127]:
sum(diff)

-701963283.9579575

In [135]:
print("The agent performs better in:", sum(diff > 0)," number of cases")

The agent performs better in: 94  number of cases


In [138]:
print("The agent performs worse in:", sum(diff < 0), " number of cases")

The agent performs worse in: 458  number of cases


In [131]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [132]:
import tensorflow as tf
import datetime, os

In [133]:
%tensorboard --logdir logs